<a href="https://www.kaggle.com/code/berrinkaradag/rating-product-sorting-reviews-in-amazon?scriptVersionId=199100439" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Case

One of the most important problems in e-commerce is the correct calculation of the scores given to the products after the sale. The solution to this problem means providing more customer satisfaction for the e-commerce site, highlighting the product for the sellers and a smooth shopping experience for the buyers. Another problem is the correct ordering of the comments given to the products. Since the prominence of misleading comments will directly affect the sales of the product, it will cause both financial loss and loss of customers. In the solution of these 2 basic problems, the e-commerce site and the sellers will increase their sales while the customers will complete the purchasing journey smoothly.

This dataset, which contains Amazon product data, includes product categories and various metadata. The product with the most comments in the electronics category has user ratings and comments.

**Task 1:** Calculate the Average Rating based on current reviews and compare with the existing average rating.

**Task 2:** Determine 20 reviews to be displayed on the product detail page for the product.

# Variables



reviewerID - ID of the reviewer

asin - ID of the product

reviewerName - name of the reviewer

helpful - helpfulness rating of the review

reviewText - text of the review

overall - rating of the product

summary - summary of the review

unixReviewTime - time of the review (unix time)

reviewTime - time of the review (raw)

day_diff - Number of days since the review

helpful_yes - Number of times the review was found helpful

total_vote - Number of votes given to the review

In [1]:
import pandas as pd

#wilsonlower bound için:
import math
import scipy.stats as st

In [2]:
df=pd.read_csv("/kaggle/input/amazon/amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


# Task 1

**Calculate the Average Rating based on current reviews and compare with the existing average rating.**

In the shared dataset, users have given points and made comments on a product. In this task, our aim is to evaluate the given points by weighting them according to date. The first average point and the weighted point according to date to be obtained must be compared.

**Step 1:** Calculate the average point of the product.

In [3]:
df["overall"].mean()

4.587589013224822

**Step 2:** Calculate the weighted average score by date.

In [4]:
def average_score (data, w1=50, w2=25, w3=15, w4=10):
    return data.loc[data["day_diff"]<=data["day_diff"].quantile(0.25), "overall"].mean() *w1/100 +\
           data.loc[(data["day_diff"]>data["day_diff"].quantile(0.25)) & (data["day_diff"]<=data["day_diff"].quantile(0.50)), "overall"].mean() *w2/100 +\
           data.loc[(data["day_diff"]>data["day_diff"].quantile(0.50)) & (data["day_diff"]<= data["day_diff"].quantile(0.75)), "overall"].mean() * w3/100 +\
           data.loc[data["day_diff"]> data["day_diff"].quantile(0.75), "overall"].mean()*w4/100

average_score(df)

4.637306192407316

**Step 3:** Compare and interpret the average of each time period in the weighted score.

In [5]:
def period (data, w1=50, w2=25, w3=15, w4=10):
    q1= data.loc[data["day_diff"]<=data["day_diff"].quantile(0.25), "overall"].mean() *w1/100 
    q2= data.loc[(data["day_diff"]>data["day_diff"].quantile(0.25)) & (data["day_diff"]<=data["day_diff"].quantile(0.50)), "overall"].mean() *w2/100 
    q3= data.loc[(data["day_diff"]>data["day_diff"].quantile(0.50)) & (data["day_diff"]<= data["day_diff"].quantile(0.75)), "overall"].mean() * w3/100
    q4= data.loc[data["day_diff"]> data["day_diff"].quantile(0.75), "overall"].mean()*w4/100
    weighted_avg = q1* w1 / 100 + q2 * w2 / 100 + q3 * w3 / 100 + q4 * w4 / 100
    
    return {
        "Q1 (Newest %25)": q1,
        "Q2": q2,
        "Q3": q3,
        "Q4 (Oldest %25)": q4,
        "Weighted Average": weighted_avg
    }

period(df)

{'Q1 (Newest %25)': 2.3478964401294498,
 'Q2': 1.1590351594439903,
 'Q3': 0.6857491856677523,
 'Q4 (Oldest %25)': 0.44462540716612375,
 'Weighted Average': 1.6110319284924977}

# Task 2

**Determine 20 reviews to be displayed on the product detail page for the product.**

**Step 1:** Generate the helpful_no variable.

• total_vote is the total number of up-downs given to a comment.

• up means helpful.

There is no helpful_no variable in the dataset, it needs to be generated from the existing variables. Find the number of votes that are not found helpful (helpful_no) by subtracting the number of helpful votes (helpful_yes) from the total number of votes (total_vote)

In [6]:
df["helpful_no"]=df["total_vote"]-df["helpful_yes"]
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0,0


**Step 2:** Calculate score_pos_neg_diff, score_average_rating and wilson_lower_bound scores and add them to the data.

score_pos_neg_diff: Useful votes - Unhelpful votes

score_average_rating: Useful votes / Total votes

wilson_lower_bound: Statistical method used to assess the reliability of ratings, especially binary (e.g., "like/dislike", "up/down") ratings. This method allows a review (or product) to be reliably ranked based on the number of positive ratings it has received. The goal is to prevent reviews with a small number of high ratings from being ranked higher than reviews with a large number of average ratings. Thus, reviews with only a few positive ratings are kept at the bottom when making a reliable ranking.

• Create scores according to score_pos_neg_diff. Then; save it in df with the name score_pos_neg_diff.

• Create scores according to score_average_rating. Then; save it in df with the name score_average_rating.

• Create scores according to wilson_lower_bound. Then; save it in df with the name wilson_lower_bound.

In [7]:
df["score_pos_nef_diff"]=df["helpful_yes"]-df["helpful_no"]
df["score_average_rating"]=df["helpful_yes"]/df["total_vote"]


def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df["wilson_lower_bound"]=df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

**Step 3:** Identify and rank the first 20 comments by wilson_lower_bound.

In [8]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_nef_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.966337,0.957544
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.948837,0.936519
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.925620,0.912139
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.852525,0.818577
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.918367,0.808109
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.0,I own it,1393545600,2014-02-28,283,60,68,8,52,0.882353,0.784651
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.805755,0.732136
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.0,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3,19,0.880000,0.700442
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.0,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37,0.784615,0.670334
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.0,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55,0.752294,0.663595
